<a href="https://colab.research.google.com/github/Lee-Minsoo-97/Decision-Modeling/blob/main/Cyprus_Potatoes_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cyprus Potatoes Pricing Optimization

## Objective and Prerequisites

This pricing optimization problem shows you how to determine the optimal price in order to maximize projected revenue.

This modeling example is at the introductory level, where we assume that you know Python and that you have some knowledge of how to build mathematical optimization models.

---
## Problem Description

You collected a sample of past price and demand information for Cyprus potatoes at a local grocery store. Based on these past experiences, the store wants to set an optimal price that can help it to maximize expected revenue. The following table lists the past price and demand on each of the month in the past year.

| Month | Price | Demand |
| --- | --- | --- |
|1	|450	|45|
|2	|300	|103|
|3	|440	|49|
|4	|360	|86|
|5	|290	|125|
|6	|450	|52|
|7	|340	|87|
|8	|370	|68|
|9	|500	|45|
|10	|490	|44|
|11	|430	|58|
|12	|390	|68|


In this example, the goal is to identify an optimal price to set so that the total revenue is maximized. This example shows how a non-linear programming model model can help the business to:

* How to set price in a scientific manner,
* How to leverage historical data to make informed decisions, and
* How to set up proper models to help the decision making process. This Jupyter Notebook is based on the MSBA SCM518 class contents.

## Model (a) Formulation - Estimating Demand Function $d(x) = a + bx$

### Indices

$i \in \{1..12\}$: Index of months

### Parameters

$p_{i}$: Price (in USD) in month $i$.

$d_{i}$: Demand (in tons) in month $i$.

### Decision Variables

$a$: Intercept of the demand function

$b$: Slope of the demand function

### Objective Function

RMSE - we want to minimize the root mean squared error

\begin{equation}
\text{Min}_{a,b} \quad \sqrt{\frac{\sum_{i \in \{1...12\}} \left( d_{i} - (a + b*p_{i})\right)^2}{12}}
\tag{0}
\end{equation}

### Constraints

NA


---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.3 MB/s eta 0:00:00


In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

In [ ]:
p=[450,	300,	440,	360,	290,	450,	340,	370,	500,	490,	430,	390]
d=[45,	103,	49,	86,	125,	52,	87,	68,	45,	44,	58,	68]
print(p)
print(d)

[450, 300, 440, 360, 290, 450, 340, 370, 500, 490, 430, 390]
[45, 103, 49, 86, 125, 52, 87, 68, 45, 44, 58, 68]


In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('cyprus potatoes pricing')

# Inputs
month = [*range(0,12)]

p=[450,	300,	440,	360,	290,	450,	340,	370,	500,	490,	430,	390]

d=[45,	103,	49,	86,	125,	52,	87,	68,	45,	44,	58,	68]

# Decisions
a = m.addVar(vtype=GRB.CONTINUOUS,lb=-GRB.INFINITY, name='Intercept')
b = m.addVar(vtype=GRB.CONTINUOUS,lb=-GRB.INFINITY, name='Slope')

# Objective: Minimize squared errors
m.setObjective(gp.quicksum((d[i] - (a + b*p[i])) * (d[i] - (a + b*p[i])) for i in month), GRB.MINIMIZE)

# Constraints: Not applicable (NA)

# Run optimization engine
m.optimize()

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x46929e2b
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+03, 6e+05]
  QObjective range [2e+01, 4e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 0 rows, 2 columns, 0 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual   

In [ ]:
#####################################################
#         Print out the solutions of the demand function
#####################################################

print(f"\n\n___Optimal parameters for linear demand function________")
print("The intercept is %3f" % (a.x))
print("The slope is %3f " % (b.x))




___Optimal parameters for linear demand function________
The intercept is 211.314675
The slope is -0.354631 


## Model (b) Formulation - Find the optimal price

### Parameters

$a$: Intercept of the demand function

$b$: Slope of the demand function

### Decision Variables

$x$: Price to charge

### Objective Function

Revenue - we want to maximize expected revenue

\begin{equation}
\text{Max}_{x} \quad x * d(x) = x * (a + b*x)
\tag{0}
\end{equation}

### Constraints

(1) $x\geq 0$ [price non-negative]


---

## Python Implementation

We now implement the second model in Python

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('cyprus potatoes pricing')

# Inputs

a=211.314675

b=-0.354631

# Decisions
x = m.addVar(vtype=GRB.CONTINUOUS, name='Price')

# Objective: Maximize revenue
m.setObjective(x*(a + b*x), GRB.MAXIMIZE)

# Constraints:

priceConstr = m.addConstr(x >= 0, name='priceConstr')

# Run optimization engine
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0x147e1e86
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 2e+02]
  QObjective range [7e-01, 7e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective 3.14791233e+04


In [ ]:
#####################################################
#         Print out the solutions of the pricing problem
#####################################################

print(f"\n\n___Optimal price for cyprus potatoes________")
print("The optimal price is %3f" % (x.x))
print("The optimal expected revenue is %3f " % (m.ObjVal))




___Optimal price for cyprus potatoes________
The optimal price is 297.935988
The optimal expected revenue is 31479.123279 


---
##  Conclusion

In this example, we addressed the optimal pricing problem. We solved this problem using two step optimization approach:
* In the first step, we estimate the demand function,
* In the second step, we find the optimal price based on the above estimated demand function

The pricing example and the models we developed here can be adapted to address more complex pricing issues by many organizations to help make informed decisions about how to set optimal prices in a scientific approach.


##  References
[1] Sixty examples of business optimization models. https://ytyimin.github.io/tart-cherry/.